In [2]:
# `ibis` a pythonic query language.

    import ibis.omniscidb, dask, intake, sqlalchemy, pandas, pyarrow as arrow, altair, h5py as hdf5

<!--    
    
    import refs
    reload(refs)
    from refs import __doc__ as refs
    import backends_over_time
    over_time = backends_over_time.df.stack().pipe(lambda df: df[df.astype(bool)]).unstack().fillna('')
    
-->

# `ibis` a pythonic query language.

    import ibis.omniscidb, dask, intake, sqlalchemy, pandas, pyarrow as arrow, altair, h5py as hdf5

<!--    
    
    import refs
    reload(refs)
    from refs import __doc__ as refs
    import backends_over_time
    over_time = backends_over_time.df.stack().pipe(lambda df: df[df.astype(bool)]).unstack().fillna('')
    
-->

In [3]:
The design of `ibis`, and other open source projects like `intake` and `dask`, begin with the pioneering 
`"blaze"` ecosystem provided interfaces to _**store, describe, query, and process**_ data.
"blaze" had ambitious goals and is now defunct, but its influences resonate throughout
the scientific python community. `dask` emerged directly from `"blaze"`; meanwhile
contributors can now be found working on projects like `ibis` and `arrow`. 

[`ibis`][ibis] took the role of providing a generic query language for data in python, that 
is independent of the computation.

In this document we'll describe the different `ibis` backend architectures and what it means to be independent of compute.

{{refs}}

The design of `ibis`, and other open source projects like `intake` and `dask`, begin with the pioneering 
`"blaze"` ecosystem provided interfaces to _**store, describe, query, and process**_ data.
"blaze" had ambitious goals and is now defunct, but its influences resonate throughout
the scientific python community. `dask` emerged directly from `"blaze"`; meanwhile
contributors can now be found working on projects like `ibis` and `arrow`. 

[`ibis`][ibis] took the role of providing a generic query language for data in python, that 
is independent of the computation.

In this document we'll describe the different `ibis` backend architectures and what it means to be independent of compute.

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997

In [6]:
## The design of [`ibis` backends][backends].

Currently, `ibis` supports __>10__ backends.
    
    >>> dir(ibis)
    [...HDFS...WebHDFS...bigquery...clickhouse...hdf5...impala...omniscidb...pandas...pyspark...spark...sql...sqlite...]
    
Each backend implementation, that can be queried with `ibis`, has one of the three following architectures.

1. Direct execution backends - `pandas and hdf5`. 
2. Expression generating backends that create `sqlalchemy` expressions.
3. String generating backends - `ibis.bigquery and ibis.omniscidb`

In the next few sections we'll unravel some of the different capiabilities of each approach.

{{refs}}

## The design of [`ibis` backends][backends].

Currently, `ibis` supports __>10__ backends.
    
    >>> dir(ibis)
    [...HDFS...WebHDFS...bigquery...clickhouse...hdf5...impala...omniscidb...pandas...pyspark...spark...sql...sqlite...]
    
Each backend implementation, that can be queried with `ibis`, has one of the three following architectures.

1. Direct execution backends - `pandas and hdf5`. 
2. Expression generating backends that create `sqlalchemy` expressions.
3. String generating backends - `ibis.bigquery and ibis.omniscidb`

In the next few sections we'll unravel some of the different capiabilities of each approach.

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997

In [7]:
## A data-driven history of `ibis` compatability.

The table below provides an annual looked at the issues tagged in `ibis` for different backends.

{{over_time.to_html()}}

We note an early focus `pandas, sqlalchemy and "impala"`, each respectively demonstrates 
direct execution, expression generating, and string generating backends.
Improvements were made to these key backends as clickhouse, spark, and postgres. Recently, 
Quansight in partnership with OmniSci, added the `ibis.omniscidb` string generating backend.
This work pushed `ibis`'s functionality to geospational data.

> Currently, there is an ongoing effort to a [SQL Server][sql-server] support. What backends what you like to
see? `dask`? `altair`?
    
{{refs}}

## A data-driven history of `ibis` compatability.

The table below provides an annual looked at the issues tagged in `ibis` for different backends.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>omnisci</th>
      <th>spark</th>
      <th>postgres</th>
      <th>bigquery</th>
      <th>pandas</th>
      <th>sqlite</th>
      <th>impala</th>
      <th>kudu</th>
      <th>geospatial</th>
      <th>clickhouse</th>
      <th>mysql</th>
      <th>sqlalchemy</th>
    </tr>
    <tr>
      <th>created_at</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2015</th>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>2</td>
      <td>8</td>
      <td>26</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>7</td>
    </tr>
    <tr>
      <th>2016</th>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>1</td>
    </tr>
    <tr>
      <th>2017</th>
      <td></td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td>3</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>1</td>
      <td></td>
      <td>1</td>
    </tr>
    <tr>
      <th>2018</th>
      <td>7</td>
      <td></td>
      <td>2</td>
      <td>7</td>
      <td>10</td>
      <td>4</td>
      <td>4</td>
      <td></td>
      <td></td>
      <td>4</td>
      <td>1</td>
      <td></td>
    </tr>
    <tr>
      <th>2019</th>
      <td>3</td>
      <td>5</td>
      <td>4</td>
      <td>3</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>9</td>
      <td>2</td>
      <td>2</td>
      <td>2</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td></td>
    </tr>
  </tbody>
</table>

We note an early focus `pandas, sqlalchemy and "impala"`, each respectively demonstrates 
direct execution, expression generating, and string generating backends.
Improvements were made to these key backends as clickhouse, spark, and postgres. Recently, 
Quansight in partnership with OmniSci, added the `ibis.omniscidb` string generating backend.
This work pushed `ibis`'s functionality to geospational data.

> Currently, there is an ongoing effort to a [SQL Server][sql-server] support. What backends what you like to
see? `dask`? `altair`?
    
[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997

In [16]:
### `ibis` direct execution.

`ibis` direct execution backends like `pandas and hdf5` operate on conventional in-memory python objects.
`pandas` is the gold standard for structured data in python, and inspires the api for `ibis`.

    pd = ibis.pandas.connect({'A': pandas.util.testing.makeDataFrame()})
    
`pd` is an `ibis` backend based off `pandas.DataFrame` objects.

    expression = pd.table('A').head()
    
`expression` is an `ibis` query, that has `expression.compile` and `expression.execute` methods.
We'll recognize the __execute__ method when we return `pandas.DataFrame`s from `ibis` expression.
The __compile__ method does not trigger any computation, rather it constructs an intermediate form
that is interpretted by a backend.

    >>> assert isinstance(expression.compile(), ibis.expr.types.TableExpr)
    
In the case of direction execution backends, the `expression` combiles to an the original `ibis` 
expression.  And the computation is carried out based on a set of recipes defined in `ibis`.

Why would I use `ibis` instead of `pandas`?

### `ibis` direct execution.

`ibis` direct execution backends like `pandas and hdf5` operate on conventional in-memory python objects.
`pandas` is the gold standard for structured data in python, and inspires the api for `ibis`.

    pd = ibis.pandas.connect({'A': pandas.util.testing.makeDataFrame()})
    
`pd` is an `ibis` backend based off `pandas.DataFrame` objects.

    expression = pd.table('A').head()
    
`expression` is an `ibis` query, that has `expression.compile` and `expression.execute` methods.
We'll recognize the __execute__ method when we return `pandas.DataFrame`s from `ibis` expression.
The __compile__ method does not trigger any computation, rather it constructs an intermediate form
that is interpretted by a backend.

    >>> assert isinstance(expression.compile(), ibis.expr.types.TableExpr)
    
In the case of direction execution backends, the `expression` combiles to an the original `ibis` 
expression.  And the computation is carried out based on a set of recipes defined in `ibis`.

Why would I use `ibis` instead of `pandas`?

In [17]:
## `ibis` expression generating backends.

    db = ibis.sqlite.connect('lahmansbaseballdb.sqlite')
    expression = db.table('halloffame').head()
    
Expression generating backends operate on [SQL] databases that interoperator with `sqlalchemy`.

    >>> assert isinstance(expression.compile(), sqlalchemy.sql.Select)
    
In the case of expression generating backends, the intermediate representation is a `sqlalchemy` object.
    
{{refs}}

## `ibis` expression generating backends.

    db = ibis.sqlite.connect('lahmansbaseballdb.sqlite')
    expression = db.table('halloffame').head()
    
Expression generating backends operate on [SQL] databases that interoperator with `sqlalchemy`.

    >>> assert isinstance(expression.compile(), sqlalchemy.sql.Select)
    
In the case of expression generating backends, the intermediate representation is a `sqlalchemy` object.
    
[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997

In [18]:
## `ibis` string generating backends.

```bash
pip install --upgrade ibis-framework[omniscidb]
```

String generating backends allow `ibis` to interface with big data systems that manage their own computation.
    
    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')
    
`omnisci` is described as a string generating backend because the intermediate representation of the
query is a flavor of SQL.

    expression = omnisci.table('upstream_reservoir').head()
    
    >>> assert isinstance(expression.compile(), str)

    
Checkout out the heroic PR by @xmnlab
Mention the labs post here.

## `ibis` string generating backends.

```bash
pip install --upgrade ibis-framework[omniscidb]
```

String generating backends allow `ibis` to interface with big data systems that manage their own computation.
    
    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')
    
`omnisci` is described as a string generating backend because the intermediate representation of the
query is a flavor of SQL.

    expression = omnisci.table('upstream_reservoir').head()
    
    >>> assert isinstance(expression.compile(), str)

Checkout out the heroic PR by @xmnlab
Mention the labs post here.

In [36]:
Huge thanks to @cpcloud and @reback for their on going support.

Omnisci backend

    

    
    
    
SQLServer backend

## String generating backends.


```bash
pip install --upgrade ibis-framework[omniscidb]
```

Checkout out the heroic PR by @xmnlab

Huge thanks to @cpcloud and @reback for their on going support.

Omnisci backend

    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')

    >>> assert isinstance(omnisci.table('upstream_reservoir').head().compile(), str)
    
    
SQLServer backend

In [33]:
.https://medium.com/@leventov/comparison-of-the-open-source-olap-systems-for-big-data-clickhouse-druid-and-pinot-8e042a5ed1c7

.https://medium.com/@leventov/comparison-of-the-open-source-olap-systems-for-big-data-clickhouse-druid-and-pinot-8e042a5ed1c7

## Who is it for?

## Why is it useful?

## Why does it exist?

## What can you learn?

https://docs.sqlalchemy.org/en/13/core/tutorial.html

In [ ]:
add geo stuff

https://github.com/ibis-project/ibis/commit/8bb84bb362ab0768c48276406e263e1f6d122d52

In [13]:
#https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py


#https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py